In [8]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def is_substring(a,b, threshold=0.7):
    """
    Проверяем, что одна строка является подстрокой у второй
    с вероятностью выше threshold.
    
    Этот метод подразумевает, что после распознавания речи слова могут быть с ошибками
    """
    long_text=a if len(a)>=len(b) else b
    short_text=b if len(a)>=len(b) else a
    
    max_sim=0
    for i in range(0,len(long_text)-len(short_text)+1):
        sim=similarity(long_text[i:i+len(short_text)],short_text)
        if sim>max_sim:
            max_sim=sim
        if max_sim>threshold:
            break
    
    return max_sim>threshold

print(is_substring("привет мир, как у вас дела?","привыт мир"))
print(is_substring("привет мир, как у вас дела?","как увас дела"))

True
True


In [6]:
class SttRow:
    def __init__(self, line):
        self.text = line.split("|")[0]
        self.start = float(line.split("|")[1])
        self.end = float(line.split("|")[2]) 
        
    def __str__(self):
        return f"SttRow(text='{self.text}', start={self.start}, end={self.end})"

    
    def get_text(self) -> str:
        return self.text
    
    def get_start(self) -> float:
        return self.start
    
    def get_end(self) -> float:
        return self.end
        

print(SttRow("я был озадачен |13.56|14.58"))

SttRow(text='я был озадачен ', start=13.56, end=14.58)


In [23]:
clean_dataset=['после этого мокер заговорил.', 
              'американский юноша стреляет в президента чтобы обратить на себя внимание малознакомой женщины.', 
              'я был озадачен.', 
              'маруся подумала — дело знакомое. а сколько мне будут платить? три доллара в час. рубинчик обещал позвонить.', 
              'на этот счёт, имеется такая запись', 'я готовил развёрнутую информацию о выставке декоративных собак.', 
              'есть мнение что он активно готовится к побегу. я промолчал.', 
              'и все-таки случай для вас неприятный, крайне неприятный, пятно на вашей журналистской репутации.', 
              'лера была в детском саду. так заведующая его одного и хвалила.', 
              'его архитектура напоминает кучу детских игрушек.']
wav2vec_result=['после этого мокер заговорил |0|2.56', 
                 'американский юноши стреляет президента чтобы обратить на себя внимание малознакомые женщины |5.44|11.18', 
                 'я был озадачен |13.56|14.58', 
                 'марусья подумала дело знакомое а сколько мне будут платить |16.54|20.82', 
                 'три доллара в час |21.38|22.5', 
                 'рубинчик обещал позвонить |23.02|24.54', 
                 'на этот счет имеется такая запись |28.36|30.759999999999998', 
                 'я говорила развернутую информацию о выставке декоративных собак |34.82|38.68', 
                 'есть мнение что он активно готовится к побегу я промолчал |43.84|47.9', 
                 'и все-таки случай для вас неприятный крайне неприятный |49.86|53.78', 
                 'пятно на вашей журналистской репутации |54.379999999999995|56.62', 
                 'лера была в детском саду |61.66|63.04', 
                 'так заведующая его одна выи хвалила |63.56|65.75999999999999', 
                 'его архитектура напоминает кучу детских игрушек |72.88000000000001|76.2', 
                 'дядя понизил голос если тебе нужны деньги скажи |81.28|84.9']

wav2vec_dataset=[]
for line in wav2vec_result:
    wav2vec_dataset.append(SttRow(line))
    

In [24]:
print(len(clean_dataset))
print(len(wav2vec_dataset))

10
15


In [44]:
import os

def aligned_text(clean_lines:[],wav2vec_lines:[])->[]:
    j_start=0
    
    aligned_result=[]

    for i in range(len(clean_lines)):
        text1=clean_lines[i]
#         print("text1:",text1)
        found=False
        found_sentences=[]
        j_current=j_start
        
        for j in range(max(0,j_current-1),min(10+j_current,len(wav2vec_lines))):
            text2=wav2vec_lines[j].get_text()
#             print("text2:",text2)
            
            subsstring=is_substring(text1,text2,0.7)
            if subsstring:
                found=True
#                 print("subsstring:",subsstring)
                found_sentences.append(wav2vec_lines[j])

            if not subsstring and found:
                j_start=j
                break
        if not found:
            j_start=j_current

        start_s=0
        k=0
        while k<len(found_sentences)-1:
            s1=found_sentences[k]
            change=False
            for m in range(k+1,len(found_sentences)):
                s2=found_sentences[m]

                if is_substring(s1.get_text(),s2.get_text(),0.7):
                    start_s=m
                    k=m
                    change=True
                    break
            if not change:
                k+=1


        first_time=None
        last_time=None
        r_s=""
        while start_s<len(found_sentences):
            if len(found_sentences[start_s].get_text().strip())=="":
                start_s+=1
                continue
            if first_time is None:
                first_time=found_sentences[start_s].get_start()
            last_time=found_sentences[start_s].get_end()

            s=found_sentences[start_s].get_text()
            r_s+=s+" "
            start_s+=1

        if r_s!="":
            aligned_result.append((f"{clean_lines[i]}",SttRow(f"{r_s}|{first_time}|{last_time}")))
    return aligned_result


In [45]:
aligned_result = aligned_text(clean_dataset, wav2vec_dataset)

for orig,found in aligned_result:
    print("_____________")
    print(orig)
    print(found)
    

_____________
после этого мокер заговорил.
SttRow(text='после этого мокер заговорил  ', start=0.0, end=2.56)
_____________
американский юноша стреляет в президента чтобы обратить на себя внимание малознакомой женщины.
SttRow(text='американский юноши стреляет президента чтобы обратить на себя внимание малознакомые женщины  ', start=5.44, end=11.18)
_____________
я был озадачен.
SttRow(text='я был озадачен  ', start=13.56, end=14.58)
_____________
маруся подумала — дело знакомое. а сколько мне будут платить? три доллара в час. рубинчик обещал позвонить.
SttRow(text='марусья подумала дело знакомое а сколько мне будут платить  три доллара в час  рубинчик обещал позвонить  ', start=16.54, end=24.54)
_____________
на этот счёт, имеется такая запись
SttRow(text='на этот счет имеется такая запись  ', start=28.36, end=30.759999999999998)
_____________
я готовил развёрнутую информацию о выставке декоративных собак.
SttRow(text='я говорила развернутую информацию о выставке декоративных собак  ', 